<a href="https://colab.research.google.com/github/bimadewantoro/indonesian-news-summarization/blob/main/Abstractive_Summarization_Indonesia_Datasheet_Endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Module

In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from colabcode import ColabCode
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

Define Flask App

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
!ngrok authtoken 2KwoaOX9u62fDd4tJisKC6QdsPu_3Gfo2Lw4s3YjU5Dc8PbQj

Load model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

Define function to preprocess text

In [5]:
def preprocess_text(text):
    text = "summarize: " + text
    return text

Define function to postprocess tokens

In [6]:
def postprocess_text(text):
    text = text.replace("</s>", "")
    text = text.replace("<pad>", "")
    text = text.strip()
    return text

Define function to Summary

In [7]:
def predict(text):
    # Preprocess text
    text = preprocess_text(text)
    # Encode text
    text = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    # Generate summary
    summary_ids = model.generate(text, num_beams=4, no_repeat_ngram_size=2, min_length=30, max_length=100, early_stopping=True)
    # Decode summary
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    # Postprocess summary
    output = postprocess_text(output)
    return output

Define route

In [8]:
# Define route
@app.route('/', methods=['GET'])
def home():
    return '''<h1>Summarize API</h1>
    <p>A prototype API for summarizing text.</p>'''

@app.route('/summary', methods=['POST'])
def summary_route():
    # Get text format
    content_type = request.headers['Content-Type']

    # Extract text based on format
    if content_type == 'application/json':
        text = request.json['text']
    elif content_type == 'application/text':
        text = request.data.decode('utf-8')
    elif content_type == 'application/x-www-form-urlencoded':
        text = request.form['text']
    else:
        return jsonify({'error': 'Invalid content type'}), 400

    # Predict summary
    summary = predict(text)
    # Return summary
    return jsonify({'summary': summary})

Run App

In [ ]:
if __name__ == '__main__':
    ColabCode(port=10000, code=False)
    app.run()